## Criando as tools como Agentes

In [1]:
from langchain_community.document_loaders import WebBaseLoader
import bs4

bs4_strainer = bs4.SoupStrainer(class_=("biotop"))

loader = WebBaseLoader(
    web_paths=("https://www.statsf1.com/en/2024/abou-dhabi.aspx",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [2]:
# Quebrando os documentos em chunks
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(docs)

In [3]:
print(f"Website post splitted into {len(documents)} pieces of documents")

Website post splitted into 71 pieces of documents


In [4]:
documents[1]

Document(metadata={'source': 'https://www.statsf1.com/en/2024/abou-dhabi.aspx'}, page_content="La guerre est déclarée entre George Russell et Max Verstappen. Huit jours plus tôt, au Qatar, le Néerlandais a été dépossédé de la pole position au profit du Britannique, sous prétexte qu'il aurait gêné celui-ci en roulant trop lentement à la fin des qualifications. Furieux, Verstappen a eu une vive altercation avec son adversaire à la sortie du collège des commissaires, puis a affirmé face caméra n'avoir «\xa0plus aucun respect\xa0» pour celui-ci. Là-bas, Russell a refusé de répondre, se contentant de mettre cette altercation au compte du caractère sanguin du pilote Red Bull. Mais à Abou Dhabi, il lâche les chiens. Dévoilant sa propre version des faits, il accuse Verstappen de l'avoir menacé verbalement. «\xa0Avant même que je dise un mot aux commissaires, il jurait contre ceux-ci, narre-t-il. Il était en colère avant même que je ne parle, mais au final, les faits sont les faits. Il allait t

### Importando o embeeding

In [5]:
import os

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = os.environ.get('OPEN_AI_KEY')
    
from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

O retriever tool é usado para recuperar informação relevante do vector database para passar pro LLM como contexto.

In [6]:
!pip install faiss-cpu

In [7]:
from langchain_community.vectorstores import FAISS
from langchain.tools.retriever import create_retriever_tool

vector = FAISS.from_documents(documents, embeddings)
retriever = vector.as_retriever()

# Criando o retriever customizado
retriever_tool = create_retriever_tool (retriever, "F1_AbuDhabi_search", """Search for information 
about 2024 Abu Dhabi Grand Prix. For any questions about 2024 Abu Dhabi Grand Prix, you must use this tool""")

Criando um segundo retriever:

In [11]:
bs4_strainer = bs4.SoupStrainer(class_=("card white rounded-0 mt-4"))

loader = WebBaseLoader(
    web_paths=("https://www.racingcircuits.info/middle-east/united-arab-emirates/yas-marina.html",),
    bs_kwargs={"parse_only": bs4_strainer},
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
documents = text_splitter.split_documents(docs)

print(f"Website post splitted into {len(documents)} pieces of documents")

print(documents[0])

vector = FAISS.from_documents(documents, embeddings)
retriever = vector.as_retriever()

retriever_tool_2 = create_retriever_tool (retriever, "F1_AbuDhabi_circuit", """Search for information 
about Yas Marina Circuit. For any questions about Yas Marina circuit, you must use this tool""")

Website post splitted into 9 pieces of documents
page_content='Circuit Overview




The Yas Marina Circuit (Arabic: حلبة مرسى ياس) is believed to be the most expensive circuit in history, having been constructed for Formula One on a man-made island at a reputed cost of $1 billion. Needless to say, it lacks nothing in terms of facilities, yet in its original form failed to capture the imagination of the drivers and fans, with overtaking in particular proving particularly difficult.
A reconfiguration in 2021 sought to improve the situation by eliminating some of the more stop-start sections and providing a more flowing layout. While designers were limited by the available space, the revamp has included the insertion of a sweeping banked corner at its southern end.
Aside from Formula One, the circuit has also hosted the short-lived FIA GT1 World Series, rounds of GP2 Asia and the full GP2 Series and overseas rounds of the V8 Supercar Championships, while 2012 saw the creation of a 12 Hour

Lista de ferramentas do agente

In [12]:
tools = [retriever_tool, retriever_tool_2] 

## Criando um agente

In [13]:
!pip install langchainhub

  Obtaining dependency information for langchainhub from https://files.pythonhosted.org/packages/35/63/40328157ddee807991f2f1992c2ad88f479b2472dc9e40d08ccf10700735/langchainhub-0.1.21-py3-none-any.whl.metadata
  Obtaining dependency information for types-requests<3.0.0.0,>=2.31.0.2 from https://files.pythonhosted.org/packages/d7/01/485b3026ff90e5190b5e24f1711522e06c79f4a56c8f4b95848ac072e20f/types_requests-2.32.0.20241016-py3-none-any.whl.metadata
  Obtaining dependency information for urllib3<3,>=1.21.1 from https://files.pythonhosted.org/packages/c8/19/4ec628951a74043532ca2cf5d97b7b14863931476d117c471e8e2b1eb39f/urllib3-2.3.0-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/128.4 kB ? eta -:--:--
   ---------------------------------------- 128.4/128.4 kB 3.7 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.8
    Uninstalling urllib3-1.26.8:
      Successfully uninstalled urllib3-1.26.8


In [15]:
from langchain import hub
prompt = hub.pull("hwchase17/openai-functions-agent") # prompt para que o llm agirá como agente

In [16]:
import os 
from langchain_openai import ChatOpenAI

if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = os.environ.get('OPEN_AI_KEY')

llm = ChatOpenAI(model="gpt-4o-mini") # modelo

In [18]:
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor

agent = create_openai_functions_agent(llm,tools,prompt)
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)

In [21]:
agent_executor.invoke({"input":"What was the weather conditions of 2024 Abu Dhabi Grand Prix? Answer it in 50 characters"})



> Entering new AgentExecutor chain...

Invoking: `F1_AbuDhabi_search` with `{'query': '2024 Abu Dhabi Grand Prix weather conditions'}`


Le Grand Prix
L'atmosphère est très douce (27°C) en ce début de soirée sur Yas Marina, mais la température de la piste est nettement plus basse que la veille (32°C au lieu de 42°C). Les pneumatiques devraient donc moins souffrir. Tous les pilotes partent en gommes médiums (C4), sauf Hamilton qui s'élance avec le composé dur (C3). Cette finale du championnat 2024 est un beau succès avec des tribunes remplies, notamment par les fans de Max Verstappen qui tiennent à fêter le quatrième sacre de leur idole.

Départ: Leclerc prend un bon envol tandis que Verstappen, très bien parti, double aussitôt Sainz, prend l'aspiration de Piastri, et se décale brusquement à l'intérieur au freinage pour surprendre ce dernier. Il est bien trop optimiste et touche la McLaren. Les deux bolides pirouettent, mais si Verstappen exécute un 360° et se relance au cœur du pelot

{'input': 'What was the weather conditions of 2024 Abu Dhabi Grand Prix? Answer it in 50 characters',
 'output': 'Weather: Mild at 27°C, track temp 32°C, clear skies.'}

In [23]:
agent_executor.invoke({"input":"What were the tyre options available for the 2024 Abu Dhabi Grand Prix?"})



> Entering new AgentExecutor chain...

Invoking: `F1_AbuDhabi_search` with `{'query': '2024 Abu Dhabi Grand Prix tyre options'}`


Le Grand Prix
L'atmosphère est très douce (27°C) en ce début de soirée sur Yas Marina, mais la température de la piste est nettement plus basse que la veille (32°C au lieu de 42°C). Les pneumatiques devraient donc moins souffrir. Tous les pilotes partent en gommes médiums (C4), sauf Hamilton qui s'élance avec le composé dur (C3). Cette finale du championnat 2024 est un beau succès avec des tribunes remplies, notamment par les fans de Max Verstappen qui tiennent à fêter le quatrième sacre de leur idole.

Départ: Leclerc prend un bon envol tandis que Verstappen, très bien parti, double aussitôt Sainz, prend l'aspiration de Piastri, et se décale brusquement à l'intérieur au freinage pour surprendre ce dernier. Il est bien trop optimiste et touche la McLaren. Les deux bolides pirouettent, mais si Verstappen exécute un 360° et se relance au cœur du peloton, Pi

{'input': 'What were the tyre options available for the 2024 Abu Dhabi Grand Prix?',
 'output': 'For the 2024 Abu Dhabi Grand Prix, the tyre options available were:\n\n- Medium (C4)\n- Hard (C3)\n\nMost drivers started the race on medium tyres, except for Lewis Hamilton, who opted for the hard compound.'}

In [24]:
agent_executor.invoke({"input":"Why was Verstappen punished during 2024 Abu Dhabi Grand Prix Race?"})



> Entering new AgentExecutor chain...

Invoking: `F1_AbuDhabi_search` with `{'query': 'Verstappen punishment 2024 Abu Dhabi Grand Prix'}`


La saison ne pouvait se clore sans une énième controverse impliquant Max Verstappen. Si le Néerlandais présente rapidement ses excuses à Oscar Piastri pour l'avoir harponné au départ, il ne digère pas la pénalité qui en a résulté pour lui. Sous le coup de l'émotion, il a qualifié par radio les commissaires de course d' « idiots stupides ». Après l'arrivée, pas question pour lui de rétropédaler: « Bien sûr, c'est de ma faute, mais je m'attendais à être puni bien plus sévèrement ! ironise-t-il. De 20, 30 secondes, ou d'un stop-and-go ! Il y a matière à discussion. Je ne comprends rien à leur logique mais, peu importe. Je ne vais pas me mettre en colère pour ça. » Christian Horner minimise l'incident: « Dans n'importe quel sport, il y aura toujours de la frustration dans le feu de l'action. Si on pouvait entendre les footballers quand ils reçoivent 

{'input': 'Why was Verstappen punished during 2024 Abu Dhabi Grand Prix Race?',
 'output': 'Max Verstappen was punished during the 2024 Abu Dhabi Grand Prix for colliding with Oscar Piastri at the start of the race. Verstappen made a bold move to overtake Piastri but miscalculated and ended up hitting the McLaren, causing both drivers to spin. As a result of this incident, Verstappen received a 10-second penalty. \n\nAfter the race, Verstappen expressed his frustration with the penalty, suggesting that he expected a harsher punishment, such as a longer time penalty or a stop-and-go penalty. He also criticized the race stewards, referring to them as "stupid idiots" in the heat of the moment. Despite acknowledging his fault in the incident, he remained puzzled by the logic behind the penalty given.'}

In [25]:
agent_executor.invoke({"input":"When the first Yas Marina Grand Prix happened?"})



> Entering new AgentExecutor chain...

Invoking: `F1_AbuDhabi_circuit` with `{'query': 'first Yas Marina Grand Prix date'}`


Circuit Overview




The Yas Marina Circuit (Arabic: حلبة مرسى ياس) is believed to be the most expensive circuit in history, having been constructed for Formula One on a man-made island at a reputed cost of $1 billion. Needless to say, it lacks nothing in terms of facilities, yet in its original form failed to capture the imagination of the drivers and fans, with overtaking in particular proving particularly difficult.
A reconfiguration in 2021 sought to improve the situation by eliminating some of the more stop-start sections and providing a more flowing layout. While designers were limited by the available space, the revamp has included the insertion of a sweeping banked corner at its southern end.
Aside from Formula One, the circuit has also hosted the short-lived FIA GT1 World Series, rounds of GP2 Asia and the full GP2 Series and overseas rounds of the V8

{'input': 'When the first Yas Marina Grand Prix happened?',
 'output': 'The first Yas Marina Grand Prix took place on November 1, 2009.'}

In [26]:
agent_executor.invoke({"input":"How many turns the Abu Dhabi circuit is comprised of?"})



> Entering new AgentExecutor chain...

Invoking: `F1_AbuDhabi_circuit` with `{'query': 'number of turns in Abu Dhabi circuit'}`


Circuit Overview




The Yas Marina Circuit (Arabic: حلبة مرسى ياس) is believed to be the most expensive circuit in history, having been constructed for Formula One on a man-made island at a reputed cost of $1 billion. Needless to say, it lacks nothing in terms of facilities, yet in its original form failed to capture the imagination of the drivers and fans, with overtaking in particular proving particularly difficult.
A reconfiguration in 2021 sought to improve the situation by eliminating some of the more stop-start sections and providing a more flowing layout. While designers were limited by the available space, the revamp has included the insertion of a sweeping banked corner at its southern end.
Aside from Formula One, the circuit has also hosted the short-lived FIA GT1 World Series, rounds of GP2 Asia and the full GP2 Series and overseas rounds of th

{'input': 'How many turns the Abu Dhabi circuit is comprised of?',
 'output': 'The Yas Marina Circuit, which hosts the Abu Dhabi Grand Prix, is comprised of 16 turns.'}

In [27]:
agent_executor.invoke({"input":"In which year Yas Marina circuit went into changes for the first time?"})



> Entering new AgentExecutor chain...

Invoking: `F1_AbuDhabi_circuit` with `{'query': 'Yas Marina Circuit changes history'}`


Circuit Overview




The Yas Marina Circuit (Arabic: حلبة مرسى ياس) is believed to be the most expensive circuit in history, having been constructed for Formula One on a man-made island at a reputed cost of $1 billion. Needless to say, it lacks nothing in terms of facilities, yet in its original form failed to capture the imagination of the drivers and fans, with overtaking in particular proving particularly difficult.
A reconfiguration in 2021 sought to improve the situation by eliminating some of the more stop-start sections and providing a more flowing layout. While designers were limited by the available space, the revamp has included the insertion of a sweeping banked corner at its southern end.
Aside from Formula One, the circuit has also hosted the short-lived FIA GT1 World Series, rounds of GP2 Asia and the full GP2 Series and overseas rounds of the 

{'input': 'In which year Yas Marina circuit went into changes for the first time?',
 'output': "The Yas Marina Circuit underwent its first significant changes in 2021. The reconfiguration aimed to improve the circuit's layout by eliminating some of the stop-start sections and creating a more flowing design."}